In [1]:
import requests, zipfile, io
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
from PIL import Image
from io import BytesIO
import random
import tensorflow as tf
import time
import tensorflow.keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense
from tensorflow.keras import optimizers
from tensorflow.keras.models import Model

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import sparse_categorical_crossentropy
from tensorflow.keras.losses import categorical_crossentropy
import requests



In [2]:
#load input
#600x1800x1800
x = np.load('xAll0.npy')
x2 = np.load('xAll1.npy')


In [3]:
x = x.reshape(-1,1800,1800, 1)
x2 = x2.reshape(-1,1800,1800, 1)
print(x.shape)

(600, 1800, 1800, 1)


In [4]:
y = np.load('y0.npy')
y2 = np.load('y1.npy')
#y = np.random.choice([0,1], size = (600, 8))
print(y.shape,y2.shape)

(600, 8) (600, 8)


In [14]:
valN = 150
valx = x2[:valN, :, :, :]
valy = y2[:valN,:]
train_labels = y
print(valx.shape)
print(valy.shape)

(150, 1800, 1800, 1)
(150, 8)


In [6]:
def get_train(x=x, y = y, lst = [1,3,6]):
    outx = []
    outy = []
    for i in lst:
        outx.append(x[i])
        outy.append(y[i])
        
    return np.asarray(outx), np.asarray(outy)
        

In [7]:
newx,newy = get_train()

In [8]:
def gen_model(reg = 3.4, lr = 5.3, model = (2, 600, 600)):
    input_tensor = tf.keras.Input(shape=(1800,1800,1) )
    r = 10**(-reg)
    learningRate = 10**(-lr)
    filter_size, FC1_size, FC2_size = model
    input_tensor = tf.keras.Input(shape=(1800,1800,1) )
    c1 = Conv2D(3, kernel_size=(filter_size, filter_size), activation='relu', 
                input_shape=(1800,1800,1), kernel_regularizer = tf.keras.regularizers.L2(r))(input_tensor) 
    l1 = tf.keras.layers.LayerNormalization()(c1)
    mp1 = tf.keras.layers.MaxPool2D()(l1)
    c2 = Conv2D(3, kernel_size=(filter_size, filter_size), activation='relu', 
                kernel_regularizer = tf.keras.regularizers.L2(r))(mp1)
    l2 = tf.keras.layers.LayerNormalization()(c2)
    mp2 = tf.keras.layers.MaxPool2D()(l2)
    c3 = Conv2D(3, kernel_size=(filter_size, filter_size), activation='relu', 
                kernel_regularizer = tf.keras.regularizers.L2(r))(mp2)
    l3 = tf.keras.layers.LayerNormalization()(c3)
    mp3 = tf.keras.layers.MaxPool2D()(l3)
    c4 = Conv2D(3, kernel_size=(filter_size, filter_size), activation='relu', 
    kernel_regularizer = tf.keras.regularizers.L2(r))(mp3)
    l4 = tf.keras.layers.LayerNormalization()(c4)
    mp4 = tf.keras.layers.MaxPool2D()(l4)
    c5 = Conv2D(3, kernel_size=(filter_size, filter_size), activation='relu', 
    kernel_regularizer = tf.keras.regularizers.L2(r))(mp4)
    l5 = tf.keras.layers.LayerNormalization()(c5)
    mp5 = tf.keras.layers.MaxPool2D()(l5)
    c6 = Conv2D(3, kernel_size=(filter_size, filter_size), activation='relu', 
    kernel_regularizer = tf.keras.regularizers.L2(r))(mp5)
    l6 = tf.keras.layers.LayerNormalization()(c6)
    mp6 = tf.keras.layers.MaxPool2D()(l6)
    head_out = Flatten()(mp6)
    FC1 = Dense(FC1_size, activation = 'relu', kernel_regularizer = tf.keras.regularizers.L2(r))(head_out)
    l7 = tf.keras.layers.LayerNormalization()(FC1)
    FC2 = Dense(FC2_size, activation = 'relu', kernel_regularizer = tf.keras.regularizers.L2(r))(l7)
    predictions = Dense(8, activation = 'sigmoid', kernel_regularizer = tf.keras.regularizers.L2(r)) (FC2)
    model = Model(inputs=input_tensor,outputs=predictions)
    #run
    model.compile(loss= categorical_crossentropy,
                  optimizer=Adam(learning_rate=learningRate),
                  metrics=['accuracy'])
    return model

In [9]:
def train(train_examples=newx,train_labels=newy, validation_data = (valx,valy), model = None):
    if model == None:
        model = gen_model()
    numEpochs = 1
    history = model.fit(train_examples,train_labels, validation_data = (valx,valy),epochs = 1)
    return history.history['val_loss'][0]


In [10]:
results = {}

In [17]:
def mtcDataShapley(x=x, y = y,valx = valx, valy= valy, N = 5, L=25):
    for i in range(N):
        results = {}
        loss = 100
        lst = []
        model = gen_model()
        while loss > 3 and len(lst)<L:
            k= np.random.randint(0,600)
            while k in lst:
                k= np.random.randint(0,600)
            lst.append(k)
            print('running with samples '+str(lst))
            newx,newy = get_train(x,y,lst)
            newloss = train(model= model)
            if len(lst)>1:
                if k not in results:
                    results[k] = []
                results[k].append(loss-newloss)
            loss = newloss
            if len(lst)%4==0:
                print('\nLIST HAS '+str(len(lst))+' SAMPLES\n')
        t = time.time()
        rdf = pd.DataFrame(results)
        rdf.to_csv('shapley/'+str(t)+'.csv')
        print('\nSAVED RESULTS for loop\n'+str(i))

In [ ]:
mtcDataShapley()

running with samples [515]
1/1 [==============================] - 29s 29s/step - loss: 3.2138 - accuracy: 0.3333 - val_loss: 3.5543 - val_accuracy: 0.1133
running with samples [515, 589]
1/1 [==============================] - 52s 52s/step - loss: 3.1712 - accuracy: 0.6667 - val_loss: 3.5459 - val_accuracy: 0.1200
running with samples [515, 589, 176]
1/1 [==============================] - 57s 57s/step - loss: 3.1178 - accuracy: 0.6667 - val_loss: 3.5384 - val_accuracy: 0.1267
running with samples [515, 589, 176, 24]
1/1 [==============================] - 57s 57s/step - loss: 3.0739 - accuracy: 0.6667 - val_loss: 3.5314 - val_accuracy: 0.1267

LIST HAS 4 SAMPLES

running with samples [515, 589, 176, 24, 566]
1/1 [==============================] - 56s 56s/step - loss: 3.0354 - accuracy: 0.6667 - val_loss: 3.5232 - val_accuracy: 0.1267
running with samples [515, 589, 176, 24, 566, 520]
1/1 [==============================] - 54s 54s/step - loss: 2.9988 - accuracy: 0.6667 - val_loss: 3.5161 

running with samples [355, 584, 381, 316, 376, 126, 190, 370, 146, 388, 332, 227, 573, 236, 453, 86, 555, 525]
1/1 [==============================] - 45s 45s/step - loss: 2.8099 - accuracy: 0.6667 - val_loss: 3.5625 - val_accuracy: 0.1200
running with samples [355, 584, 381, 316, 376, 126, 190, 370, 146, 388, 332, 227, 573, 236, 453, 86, 555, 525, 321, 479, 330, 432, 62, 285]
1/1 [==============================] - 44s 44s/step - loss: 2.7915 - accuracy: 0.6667 - val_loss: 3.5583 - val_accuracy: 0.1200

LIST HAS 24 SAMPLES

running with samples [355, 584, 381, 316, 376, 126, 190, 370, 146, 388, 332, 227, 573, 236, 453, 86, 555, 525, 321, 479, 330, 432, 62, 285, 314]
1/1 [==============================] - 45s 45s/step - loss: 2.7739 - accuracy: 0.6667 - val_loss: 3.5535 - val_accuracy: 0.1267

SAVED RESULTS for loop
1
running with samples [235]
1/1 [==============================] - 45s 45s/step - loss: 3.7128 - accuracy: 0.0000e+00 - val_loss: 3.3894 - val_accuracy: 0.2400
running with 